In [ ]:
import sys
print(sys.executable)
print(sys.path)

In [ ]:
import findspark
findspark.init() #This line initializes findspark.

In [2]:
import os
os.environ['SPARK_HOME'] = "/opt/homebrew/Cellar/apache-spark/3.5.4/libexec" #Or your spark home.
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
#from pyspark.sql import types as T
from pyspark.sql.functions import col
from pyspark.sql import types
#from deep_translator import GoogleTranslator

In [4]:
credentials_location = './gcs.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location) \
    .set("spark.driver.extraClassPath", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.executor.extraClassPath", "./lib/gcs-connector-hadoop3-2.2.5.jar")

In [5]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/03/20 22:36:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [52]:
spark.stop()

In [6]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [7]:
df_courses = spark.read.option("header", "true").parquet('gs://jugnu-france-course-enrollments/courses_enrol_data_2025_03_20/courses_raw_parquet/*.parquet')

In [8]:
df_courses.show(5)

25/03/20 22:36:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+------------+----------------+--------------------+----------------+---------------+---------+----------------------+-------------------------------+----------------+----------------+----------------+----------------+----------------+--------------------------------+-----------+-----------+-----------+-----------+-----------+--------------------+-------------+-------------+----------+----------+----------+---------------+--------------+--------------------+------------------+--------------------+--------------------+--------------------+----------------------------+---------+-----------------+---------------------+-----------------------+-----------------------+------------------------+-----------------+-----------------+------------------+----------------+-----------+-----------------+-----------------+
|date_extract|      nom_of| nom_departement|          nom_region|type_referentiel|code_inventaire|code_rncp|intitule_certification|libelle_niveau_sortie_formation|code_fo

In [37]:
df_courses.schema

StructType([StructField('date_extract', StringType(), True), StructField('provider', StringType(), True), StructField('department', StringType(), True), StructField('region', StringType(), True), StructField('type_referentiel', StringType(), True), StructField('code_inventaire', LongType(), True), StructField('code_rncp', LongType(), True), StructField('certification_title', StringType(), True), StructField('training_exit_level', StringType(), True), StructField('code_formacode_1', DoubleType(), True), StructField('code_formacode_2', DoubleType(), True), StructField('code_formacode_3', DoubleType(), True), StructField('code_formacode_4', DoubleType(), True), StructField('code_formacode_5', DoubleType(), True), StructField('main_formacode_desc', StringType(), True), StructField('code_rome_1', StringType(), True), StructField('code_rome_2', StringType(), True), StructField('code_rome_3', StringType(), True), StructField('code_rome_4', StringType(), True), StructField('code_rome_5', Strin

In [ ]:
schema = types.StructType([
    types.StructField('date_extract', types.StringType(), True), 
    types.StructField('nom_of', types.StringType(), True), 
    types.StructField('nom_departement', types.StringType(), True), 
    types.StructField('nom_region', types.StringType(), True), 
    types.StructField('type_referentiel', types.StringType(), True), 
    types.StructField('code_inventaire', types.LongType(), True), 
    types.StructField('code_rncp', types.LongType(), True), 
    types.StructField('intitule_certification', types.StringType(), True), 
    types.StructField('libelle_niveau_sortie_formation', types.StringType(), True), 
    types.StructField('code_formacode_1', types.DoubleType(), True), 
    types.StructField('code_formacode_2', types.DoubleType(), True), 
    types.StructField('code_formacode_3', types.DoubleType(), True), 
    types.StructField('code_formacode_4', types.DoubleType(), True), 
    types.StructField('code_formacode_5', types.DoubleType(), True), 
    types.StructField('libelle_code_formacode_principal', types.StringType(), True), 
    types.StructField('code_rome_1', types.StringType(), True), 
    types.StructField('code_rome_2', types.StringType(), True), 
    types.StructField('code_rome_3', types.StringType(), True), 
    types.StructField('code_rome_4', types.StringType(), True), 
    types.StructField('code_rome_5', types.StringType(), True), 
    types.StructField('libelle_nsf_1', types.StringType(), True), 
    types.StructField('libelle_nsf_2', types.StringType(), True), 
    types.StructField('libelle_nsf_3', types.StringType(), True), 
    types.StructField('code_nsf_1', types.StringType(), True), 
    types.StructField('code_nsf_2', types.StringType(), True), 
    types.StructField('code_nsf_3', types.StringType(), True), 
    types.StructField('code_certifinfo', types.StringType(), True), 
    types.StructField('siret', types.StringType(), True), 
    types.StructField('numero_formation', types.StringType(), True), 
    types.StructField('intitule_formation', types.StringType(), True), 
    types.StructField('points_forts', types.StringType(), True), 
    types.StructField('objectif_formation', types.StringType(), True), 
    types.StructField('contenu_formation', types.StringType(), True), 
    types.StructField('resultats_attendus_formation', types.StringType(), True), 
    types.StructField('nb_action', types.IntegerType(), True), 
    types.StructField('nb_session_active', types.IntegerType(), True), 
    types.StructField('nb_session_a_distance', types.IntegerType(), True), 
    types.StructField('nombre_heures_total_min', types.IntegerType(), True), 
    types.StructField('nombre_heures_total_max', types.IntegerType(), True), 
    types.StructField('nombre_heures_total_mean', types.IntegerType(), True), 
    types.StructField('frais_ttc_tot_min', types.FloatType(), True), 
    types.StructField('frais_ttc_tot_max', types.FloatType(), True), 
    types.StructField('frais_ttc_tot_mean', types.FloatType(), True), 
    types.StructField('code_departement', types.StringType(), True), 
    types.StructField('code_region', types.StringType(), True), 
    types.StructField('nbaction_nbheures', types.IntegerType(), True), 
    types.StructField('coderegion_export', types.StringType(), True)
    ])

In [32]:
df_courses_s = spark.read.option("header", "true").schema(schema).parquet('gs://jugnu-france-course-enrollments/courses_data/courses_raw_parquet/*.parquet')

In [40]:
# Cast the 'code_inventaire' column to a string
df_courses = df_courses.withColumn("code_inventaire", df_courses["code_inventaire"].cast(types.StringType()))\
    .withColumn("code_rncp", df_courses["code_rncp"].cast(types.StringType()))\
    .withColumn("code_formacode_1", df_courses["code_formacode_1"].cast(types.StringType()))\
    .withColumn("code_formacode_2", df_courses["code_formacode_2"].cast(types.StringType()))\
    .withColumn("code_formacode_3", df_courses["code_formacode_3"].cast(types.StringType()))\
    .withColumn("code_formacode_4", df_courses["code_formacode_4"].cast(types.StringType()))\
    .withColumn("code_formacode_5", df_courses["code_formacode_5"].cast(types.StringType()))\
    .withColumn("code_nsf_1", df_courses["code_nsf_1"].cast(types.StringType()))\
    .withColumn("code_nsf_2", df_courses["code_nsf_2"].cast(types.StringType()))\
    .withColumn("code_nsf_3", df_courses["code_nsf_3"].cast(types.StringType()))\
    .withColumn("code_certifinfo", df_courses["code_certifinfo"].cast(types.StringType()))\
    .withColumn("provider_ID", df_courses["provider_ID"].cast(types.StringType()))

In [41]:
df_courses.head(2)

[Row(date_extract='2025-03-20', provider='BT FORMATION', department="Côtes-d'Armor", region='Bretagne', type_referentiel='RS', code_inventaire='6965', code_rncp='-1', certification_title='Créer et gérer des sites web avec WordPress (Tosa)', training_exit_level=None, code_formacode_1=None, code_formacode_2=None, code_formacode_3=None, code_formacode_4=None, code_formacode_5=None, main_formacode_desc=None, code_rome_1=None, code_rome_2=None, code_rome_3=None, code_rome_4=None, code_rome_5=None, nsf_code_1_desc="Informatique, traitement de l'information, réseaux de transmission", nsf_code_2_desc=None, nsf_code_3_desc=None, code_nsf_1='326.0', code_nsf_2=None, code_nsf_3=None, code_certifinfo='-1', provider_ID='48121600000044', training_ID='48121600000044_11172-60', title='LOGICIEL WORDPRESS', strengths='<p>24 ans d\'expérience à votre service.</p><p>- Durée, rythme selon vos disponibilités.</p><p>- Préparation et passage à la certification inclus.</p><h4>Intéressé(e) ? "Demander un devis"

In [42]:
# Convert the string to a timestamp with the correct format
df_courses_data = df_courses.withColumn('date_extract', F.to_date(F.col('date_extract'), 'yyyy-MM-dd'))


In [43]:
df_courses_data.head(2)

[Row(date_extract=datetime.date(2025, 3, 20), provider='BT FORMATION', department="Côtes-d'Armor", region='Bretagne', type_referentiel='RS', code_inventaire='6965', code_rncp='-1', certification_title='Créer et gérer des sites web avec WordPress (Tosa)', training_exit_level=None, code_formacode_1=None, code_formacode_2=None, code_formacode_3=None, code_formacode_4=None, code_formacode_5=None, main_formacode_desc=None, code_rome_1=None, code_rome_2=None, code_rome_3=None, code_rome_4=None, code_rome_5=None, nsf_code_1_desc="Informatique, traitement de l'information, réseaux de transmission", nsf_code_2_desc=None, nsf_code_3_desc=None, code_nsf_1='326.0', code_nsf_2=None, code_nsf_3=None, code_certifinfo='-1', provider_ID='48121600000044', training_ID='48121600000044_11172-60', title='LOGICIEL WORDPRESS', strengths='<p>24 ans d\'expérience à votre service.</p><p>- Durée, rythme selon vos disponibilités.</p><p>- Préparation et passage à la certification inclus.</p><h4>Intéressé(e) ? "Dema

In [44]:
# Assuming your DataFrame is named df

filter_values = [
    "Informatique, traitement de l'information, réseaux de transmission",
    "Enseignement, formation",
    "Commerce, vente",
    "Comptabilite, gestion",
    "Spécialités pluri-scientifiques",
    "Spécialites plurivalentes de la communication et de l'information"
]

In [45]:
df_courses_data = df_courses_data.filter(col("libelle_nsf_1").isin(filter_values))

In [46]:
df_courses.count()

68114

In [47]:
# Define the columns to rename and their new names
columns_to_rename = {
    'nom_of': 'provider',
    'siret': 'provider_ID',
    'nom_region': 'region',
    'nom_departement': 'department',
    'intitule_certification': 'certification_title',
    'libelle_niveau_sortie_formation': 'training_exit_level',
    'libelle_code_formacode_principal': 'main_formacode_desc',
    'libelle_nsf_1': 'nsf_code_1_desc',
    'libelle_nsf_2': 'nsf_code_2_desc',
    'libelle_nsf_3': 'nsf_code_3_desc',
    'numero_formation': 'training_ID',
    'intitule_formation': 'title',
    'points_forts': 'strengths',
    'nb_session_active': 'nb_active_session',
    'nb_session_a_distance': 'nb_distant_session',
    'nombre_heures_total_min': 'duration_min',
    'nombre_heures_total_max': 'duration_max',
    'nombre_heures_total_mean': 'duration_mean',
    'frais_ttc_tot_min': 'cost_min',
    'frais_ttc_tot_max': 'cost_max',
    'frais_ttc_tot_mean': 'cost_mean'
}

In [49]:
# Rename the columns
for old_name, new_name in columns_to_rename.items():
    if old_name in df_courses_data.columns:
        df_courses_data = df_courses_data.withColumnRenamed(old_name, new_name)
    else:
        print(f"Column '{old_name}' not found, skipping rename.")

Column 'nom_of' not found, skipping rename.
Column 'siret' not found, skipping rename.
Column 'nom_region' not found, skipping rename.
Column 'nom_departement' not found, skipping rename.
Column 'intitule_certification' not found, skipping rename.
Column 'libelle_niveau_sortie_formation' not found, skipping rename.
Column 'libelle_code_formacode_principal' not found, skipping rename.
Column 'libelle_nsf_1' not found, skipping rename.
Column 'libelle_nsf_2' not found, skipping rename.
Column 'libelle_nsf_3' not found, skipping rename.
Column 'numero_formation' not found, skipping rename.
Column 'intitule_formation' not found, skipping rename.
Column 'points_forts' not found, skipping rename.
Column 'nb_session_active' not found, skipping rename.
Column 'nb_session_a_distance' not found, skipping rename.
Column 'nombre_heures_total_min' not found, skipping rename.
Column 'nombre_heures_total_max' not found, skipping rename.
Column 'nombre_heures_total_mean' not found, skipping rename.
Co

In [20]:
df_courses.columns

['date_extract',
 'provider',
 'department',
 'region',
 'type_referentiel',
 'code_inventaire',
 'code_rncp',
 'certification_title',
 'training_exit_level',
 'code_formacode_1',
 'code_formacode_2',
 'code_formacode_3',
 'code_formacode_4',
 'code_formacode_5',
 'main_formacode_desc',
 'code_rome_1',
 'code_rome_2',
 'code_rome_3',
 'code_rome_4',
 'code_rome_5',
 'nsf_code_1_desc',
 'nsf_code_2_desc',
 'nsf_code_3_desc',
 'code_nsf_1',
 'code_nsf_2',
 'code_nsf_3',
 'code_certifinfo',
 'provider_ID',
 'training_ID',
 'title',
 'strengths',
 'objectif_formation',
 'contenu_formation',
 'resultats_attendus_formation',
 'nb_action',
 'nb_active_session',
 'nb_distant_session',
 'duration_min',
 'duration_max',
 'duration_mean',
 'cost_min',
 'cost_max',
 'cost_mean',
 'code_departement',
 'code_region',
 'nbaction_nbheures',
 'coderegion_export']

In [50]:
df_courses_data.schema

StructType([StructField('date_extract', DateType(), True), StructField('provider', StringType(), True), StructField('department', StringType(), True), StructField('region', StringType(), True), StructField('type_referentiel', StringType(), True), StructField('code_inventaire', StringType(), True), StructField('code_rncp', StringType(), True), StructField('certification_title', StringType(), True), StructField('training_exit_level', StringType(), True), StructField('code_formacode_1', StringType(), True), StructField('code_formacode_2', StringType(), True), StructField('code_formacode_3', StringType(), True), StructField('code_formacode_4', StringType(), True), StructField('code_formacode_5', StringType(), True), StructField('main_formacode_desc', StringType(), True), StructField('code_rome_1', StringType(), True), StructField('code_rome_2', StringType(), True), StructField('code_rome_3', StringType(), True), StructField('code_rome_4', StringType(), True), StructField('code_rome_5', Str

In [51]:
df_courses_data.coalesce(1).write.parquet('gs://jugnu-france-course-enrollments/courses_enrol_data_2025_03_20/courses_filtered', mode='overwrite')

S.unique_row_id, S.date_extract, S.provider, S.department, S.region, S.type_referential, S.code_rs, S.code_rncp, S.certification_title, S.training_exit_level, S.code_formacode_1, S.code_formacode_2, S.code_formacode_3, S.code_formacode_4, S.code_formacode_5, S.main_formacode_desc, S.code_rome_1, S.code_rome_2, S.code_rome_3, S.code_rome_4, S.code_rome_5, S.nsf_code_1_desc, S.nsf_code_2_desc, S.nsf_code_3_desc, S.nsf_code_1, S.nsf_code_2, S.nsf_code_3, S.code_certification, S.provider_id, S.training_id, S.training_title, S.strong_points, S.training_objective, S.training_content, S.expected_training_results, S.trianing_module_count, S.nb_session_active, S.nb_remote_session, S.duration_min, S.duration_max, S.duration_mean, S.fees_min, S.fees_max, S.fees_mean, S.department_code, S.region_code, S.nb_hours, S.coderegion_export

# earlier

In [ ]:
def translate_batch(unique_values):
    translations = {}
    translator = GoogleTranslator(source='auto', target='en')
    for value in unique_values:
        if value is not None:
            try:
                translations[value] = translator.translate(value)
            except Exception as e:
                print(f"Translation error: {e}")
                translations[value] = None
        else:
            translations[value] = None
    return translations

In [ ]:
columns_to_translate = ['certification_title'] #, 'title', 'main_formacode_desc']
translation_maps = {}

In [ ]:
print(f"Number of partitions: {df_courses.rdd.getNumPartitions()}")

In [ ]:
df_courses = df_courses.repartition(24)

In [ ]:
for col_name in columns_to_translate:
    unique_values = [row[0] for row in df_courses.select(col_name).distinct().collect()]
    translation_maps[col_name] = translate_batch(unique_values)

In [ ]:
for col_name in columns_to_translate:
    translation_map = translation_maps[col_name]
    broadcasted_map = sc.broadcast(translation_map)

    @F.udf(returnType=T.StringType())
    def lookup_translation(input_value):
        return broadcasted_map.value.get(input_value)

    df_courses = df_courses.withColumn(f'{col_name}_en', lookup_translation(F.col(col_name)))

In [ ]:
@F.udf(returnType=T.StringType())
def translate(input):
    if input is None:
        return None  # Or return "" if you prefer an empty string
    try:
        return GoogleTranslator(source='auto', target='en').translate(input)
    except NotValidPayload:
        return None #Or some other error handling.
    except Exception as e:
        print(f"Translation error: {e}")
        return None

In [ ]:
df_courses = df_courses\
            .withColumn('certification_title_en', translate(F.col('certification_title')))\
            .withColumn('title_en', translate(F.col('title')))\
            .withColumn('main_formacode_desc_en', translate(F.col('main_formacode_desc')))

In [ ]:
df_courses.show(5)

In [ ]:
df_courses\
    .write\
    .parquet('gs://jugnu-france-course-enrollments/courses_data/courses_raw_parquet/france_courses_en.parquet', mode='overwrite')

In [ ]:
import pandas as pd
import gcsfs
import os

def read_parquet_from_gcs(gcs_path, credentials_path):
    """Reads a Parquet file from Google Cloud Storage into a pandas DataFrame."""
    try:
        # Set the environment variable within the function
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcs.json"

        fs = gcsfs.GCSFileSystem()
        with fs.open(gcs_path) as f:
            df = pd.read_parquet(f, )
        return df
    except Exception as e:
        print(f"Error reading Parquet file: {e}")
        return None

# Example usage:
gcs_path = "gs://your-bucket/path/to/your/file.parquet"  # Replace with your GCS path
credentials_path = "gcs.json"  # Replace with your path

df = read_parquet_from_gcs(gcs_path, credentials_path)

if df is not None:
    print(df.head())